In [1]:
# Import Libraries
from rdflib import Graph
import pandas as pd
import owlrl

In [2]:
#Functions needed for the saving and the reasoning of the graph

def saveGraph(graph, file_output):

    graph.serialize(destination=file_output, format='ttl')
    print("Triples including ontology: '" + str(len(graph)) + "'.")

def performReasoning(graph, ontology_file):

    print("Triples including ontology: '" + str(len(graph)) + "'.")
    
    #We should load the ontology first
    graph.load(ontology_file,  format='ttl') #e.g., format=ttl


    #We apply reasoning and expand the graph with new triples 
    owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(graph)

    print("Triples after OWL 2 RL reasoning: '" + str(len(graph)) + "'.")

In [ ]:
#Initialise a graph
g = Graph()

# Load the asserted tripples from the KG
g.parse("cw-data.ttl", format="ttl")
print("Loaded '" + str(len(g)) + "' triples.")

#perform the reasoning  with the created ontology and save to a new KG in ttl format
performReasoning(g,'zdetor.ttl')
saveGraph(g, 'cw-data-reasoned.ttl')

In [3]:
# Same as above but skipping the reasoning part (i.e. loaded the ttl after the reasoning)
g = Graph()
g.parse("cw-data-reasoned.ttl", format="ttl")
print("Loaded '" + str(len(g)) + "' triples.")

Loaded '77867' triples.


## Subtask SPARQL.2 
Return all the details of the restaurants that sell pizzas without tomate (i.e., pizza bianca). Return the results as a CSV file (20%).

In [4]:
qres = g.query(
"""
SELECT ?pizzaName ?description ?name ?addressLine
WHERE{
        ?pizza a zdetor:Pizza;
                zdetor:name ?pizzaName;
                zdetor:isMenuItemOf ?restaurant;
                zdetor:description ?description.
        ?restaurant zdetor:name ?name;
                    zdetor:hasAddress [ zdetor:addressLine ?addressLine] .
    FILTER (regex(LCASE(?pizzaName), \"white\") || regex(LCASE(?pizzaName), \"bianca\") || regex(LCASE(?pizzaName), \"bianco\") ).
#    FILTER NOT EXISTS {
#        FILTER regex(?description, \"tomato\").
#    }
}

""")

print(len(qres))

#Single row with one boolean vale
temp = []
for row in qres:
    element = {}
    element['name'] = str(row.name)
    element['addressLine'] = str(row.addressLine)
    element['pizzaName'] = str(row.pizzaName)
    element['description'] = str(row.description)
    temp.append(element)

new_temp = pd.DataFrame(temp)
new_temp.to_csv("Results_4.csv")
# new_temp

70


## Subtask SPARQL.3 
Return the average prize of a Margherita pizza (20%).

In [5]:
qres = g.query(
"""
SELECT (sum(?price)/count(?pizza) as ?averagePrice)
WHERE{
        ?pizza  a zdetor:MargheritaPizza;
                zdetor:price ?price;
                zdetor:isMenuItemOf [ zdetor:hasAddress [ zdetor:addressLine ?addressLine] ] .
}
#GROUP BY ?pizza ?addressLine

""")

print(len(qres))

# #Single row with one boolean vale
for row in qres:
    print(f"The average price for the Margherita Pizza is: ${str(round(float(row.averagePrice),2))}")

1
The average price for the Margherita Pizza is: $12.6


## Subtask SPARQL.4 
Return number of restaurants by city, sorted by state and number
of restaurants (20%).

In [6]:
qres = g.query(
"""
SELECT ?cityName ?state (count(?restaurant) as ?num_of_restaurants)
WHERE{
        ?restaurant a zdetor:Restaurant;
                    zdetor:hasAddress ?address .
        ?address    zdetor:hasCity ?city .
        ?city zdetor:name ?cityName .
        
        OPTIONAL {?address  zdetor:hasState [ zdetor:name ?state] .}
}
GROUP BY ?cityName
ORDER BY ?state ?cityName

""")

print(len(qres))

#Single row with one boolean vale
temp = []
for row in qres:
    element = {}
    element['num_of_restaurants'] = str(row.num_of_restaurants)
    element['cityName'] = str(row.cityName)
    element['state'] = str(row.state)
    temp.append(element)

new_temp = pd.DataFrame(temp)
new_temp.to_csv("Results_4.csv")
# new_temp

674


## Subtask SPARQL.5 Return the list of restaurants with missing postcode (20%).

In [7]:
qres = g.query(
"""
SELECT ?restaurant ?restaurantName ?postCode
WHERE{
        ?restaurant a zdetor:Restaurant;
                    zdetor:hasAddress ?address ;
                    zdetor:name ?restaurantName .
        ?address    zdetor:hasCity ?city .
        OPTIONAL {?address  zdetor:postCode ?postCode .}
        
        FILTER( !bound( ?postCode ) )
}

""")

print(len(qres))

#Single row with one boolean vale
temp = []
for row in qres:
    element = {}
    element['restaurant'] = str(row.restaurant)
    element['restaurantName'] = str(row.restaurantName)
    element['postCode'] = str(row.postCode)
    temp.append(element)

new_temp = pd.DataFrame(temp)
new_temp.to_csv("Results_5.csv")
# new_temp

14
